# Лабораторна робота 6
### Студента групи МІТ-31 (підгрупа 2)
### Шило Івана Костянтиновича
## Завдання

1. Виконати вирішення задач класифікації для 3 класів з набору даних food101
2. Індекси класів визначити індивідуально за залежностями: i1=n-1,i2=n+29,i3=n+59 (де і1,і2,і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n - номер за списком (обчислені значення індексів вказані у даному документі у стовпчиках D:F) )
3. Отримані результати викласти на github у репозиторій ml2021 в основну (default) гілку в папці Lab6.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import pathlib
import numpy as np
import pandas as pd
import zipfile
import os

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
# Unzip file
zip_ref = zipfile.ZipFile("small.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [3]:
main_dir="small"

In [4]:
for dirpath, dirnames, filenames in os.walk(main_dir):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 3 directories and 0 images in 'small'.
There are 3 directories and 0 images in 'small/test'.
There are 0 directories and 250 images in 'small/test/gnocchi'.
There are 0 directories and 250 images in 'small/test/pork_chop'.
There are 0 directories and 250 images in 'small/test/cheesecake'.
There are 1 directories and 1 images in 'small/__MACOSX'.
There are 1 directories and 2 images in 'small/__MACOSX/101_food_classes_10_percent'.
There are 0 directories and 34 images in 'small/__MACOSX/101_food_classes_10_percent/test'.
There are 3 directories and 0 images in 'small/train'.
There are 0 directories and 75 images in 'small/train/gnocchi'.
There are 0 directories and 75 images in 'small/train/pork_chop'.
There are 0 directories and 75 images in 'small/train/cheesecake'.


In [5]:
data_dir = pathlib.Path("small/train/") # turn our training path into a Python path
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) # created a list of class_names from the subdirectories
print(class_names)

['cheesecake' 'gnocchi' 'pork_chop']


In [6]:
def view_random_image(target_dir, target_class):
      # Setup target directory (we'll view images from here)
  target_folder = target_dir+target_class

  # Get a random image path
  random_image = random.sample(os.listdir(target_folder), 1)

  # Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape: {img.shape}") # show the shape of the image

  return img

  # View a random image from the training dataset
img = view_random_image(target_dir="small/train/",
                        target_class="cheesecake")

Image shape: (512, 512, 3)


In [7]:
img

array([[[203, 190, 208],
        [205, 194, 211],
        [205, 194, 210],
        ...,
        [  6,   8,  29],
        [  5,  10,  30],
        [  8,  13,  33]],

       [[203, 192, 209],
        [206, 195, 212],
        [206, 195, 211],
        ...,
        [  8,  10,  31],
        [  8,  10,  31],
        [  7,  12,  32]],

       [[199, 190, 207],
        [201, 193, 208],
        [202, 194, 209],
        ...,
        [ 11,  13,  34],
        [ 10,  12,  33],
        [  8,  10,  31]],

       ...,

       [[238, 205, 126],
        [239, 206, 129],
        [238, 205, 128],
        ...,
        [100,  88,  72],
        [100,  88,  72],
        [101,  89,  75]],

       [[249, 212, 132],
        [247, 211, 133],
        [242, 209, 132],
        ...,
        [100,  88,  72],
        [ 99,  87,  71],
        [ 99,  87,  73]],

       [[247, 210, 130],
        [248, 211, 131],
        [244, 208, 130],
        ...,
        [100,  88,  72],
        [ 99,  87,  73],
        [100,  88,  74]]

In [8]:
img/255.

array([[[0.79607843, 0.74509804, 0.81568627],
        [0.80392157, 0.76078431, 0.82745098],
        [0.80392157, 0.76078431, 0.82352941],
        ...,
        [0.02352941, 0.03137255, 0.11372549],
        [0.01960784, 0.03921569, 0.11764706],
        [0.03137255, 0.05098039, 0.12941176]],

       [[0.79607843, 0.75294118, 0.81960784],
        [0.80784314, 0.76470588, 0.83137255],
        [0.80784314, 0.76470588, 0.82745098],
        ...,
        [0.03137255, 0.03921569, 0.12156863],
        [0.03137255, 0.03921569, 0.12156863],
        [0.02745098, 0.04705882, 0.1254902 ]],

       [[0.78039216, 0.74509804, 0.81176471],
        [0.78823529, 0.75686275, 0.81568627],
        [0.79215686, 0.76078431, 0.81960784],
        ...,
        [0.04313725, 0.05098039, 0.13333333],
        [0.03921569, 0.04705882, 0.12941176],
        [0.03137255, 0.03921569, 0.12156863]],

       ...,

       [[0.93333333, 0.80392157, 0.49411765],
        [0.9372549 , 0.80784314, 0.50588235],
        [0.93333333, 0

In [9]:
# Set the seed
tf.random.set_random_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = main_dir+"/train/"
test_dir = main_dir+"/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="sparse", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="sparse",
                                               seed=42)



model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="softmax")
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=2,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 225 images belonging to 3 classes.
Found 750 images belonging to 3 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
8/8 [==============================] - 159s 20s/step - loss: -1.3521e-09 - acc: 0.3333 - val_loss: 0.1334 - val_acc: 0.3333
Epoch 2/2
8/8 [==============================] - 162s 20s/step - loss: -1.7160 - acc: 0.3333 - val_loss: 0.0267 - val_acc: 0.3333


In [10]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 10)      280       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 10)      910       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 10)      910       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 10)      910       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 10)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28090)             0         
__________

In [11]:
pd.DataFrame(history_1.history).plot(xlabel="epochs", 
ylabel="loss",title="History ins_model", xlim=(0,1))

In [12]:
def plot_loss_curves(history_1):
    """
    Returns separate loss curves for training and validation metrics.
    """ 
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [13]:
plot_loss_curves(history_1)

NameError: name 'history' is not defined

In [14]:
model_1.save('model.h5')